# 🏢 Section 18: Domain Information API Testing

**Comprehensive testing of VeevaVault Domain Information capabilities**

Domain Information endpoints allow Admins to retrieve information about the domain of the authenticated Vault and domains accessible to the authenticated user. This includes vault listings, domain types, application information, and vault family details.

## 🎯 Testing Scope

### Core Domain Information Endpoints
- **GET** `/api/{version}/objects/domain` - Retrieve Domain Information (current domain)
- **GET** `/api/{version}/objects/domain?include_application=true` - Include application details
- **GET** `/api/{version}/objects/domains` - Retrieve All Accessible Domains

### Key Features Tested
- ✅ **Domain Details** - Get current domain name, type, and vault listings
- ✅ **Vault Information** - Retrieve vault IDs, names, status, and families
- ✅ **Application Details** - Get vault application types and labels
- ✅ **Multi-Domain Access** - List all accessible domains for the user
- ✅ **Domain Types** - Validate Production, Sandbox, Demo, Test classifications
- ✅ **Vault Status** - Check Active/Inactive vault states

### Implementation Notes
Based on successful Section 15, 16, and 17 patterns:
- Uses `VaultClient` with proven authentication flow
- Implements read-only operations for safety
- Tests both basic and enhanced parameter variations
- Provides comprehensive domain and vault analysis

---

In [1]:
# 📦 Domain Information Testing Framework Setup
import sys
import os
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')

from veevavault.client.vault_client import VaultClient
from veevavault.services.queries.query_service import QueryService
import time
from datetime import datetime
from dataclasses import dataclass
from typing import List, Dict, Optional, Any
import json

# Enhanced test result tracking
@dataclass
class APITestResult:
    """Enhanced test result with detailed tracking"""
    name: str
    success: bool
    duration: float
    error_message: Optional[str] = None
    response_data: Optional[Dict] = None
    endpoint: Optional[str] = None
    method: str = "GET"
    status_code: Optional[int] = None

class DomainInformationAPITester:
    """Domain Information API Testing Framework for VeevaVault"""
    
    def __init__(self, client: VaultClient):
        self.client = client
        self.query_service = None  # Will be initialized after authentication
        self.results: List[APITestResult] = []
        self.start_time = time.time()
        
        # Domain Information specific tracking
        self.current_domain_info = None
        self.accessible_domains = []
        self.vault_applications_found = []
        self.vault_families_found = []
        self.domain_types_found = []
        self.total_api_calls_tested = 0
        
    def init_services(self):
        """Initialize services after authentication"""
        self.query_service = QueryService(self.client)
        
    def log_test(self, name: str, success: bool, duration: float, 
                 error_msg: str = None, response_data: Dict = None, 
                 endpoint: str = None, method: str = "GET", status_code: int = None):
        """Log test result with enhanced tracking"""
        result = APITestResult(
            name=name, success=success, duration=duration,
            error_message=error_msg, response_data=response_data,
            endpoint=endpoint, method=method, status_code=status_code
        )
        self.results.append(result)
        
        # Real-time feedback
        status = "✅ PASS" if success else "❌ FAIL"
        print(f"{status} [{len(self.results):02d}]: {name} ({duration:.2f}s)")
        if error_msg:
            print(f"        ❌ Error: {error_msg}")
        elif response_data and success:
            # Show key metrics for successful calls
            if 'responseStatus' in response_data:
                print(f"        📊 Status: {response_data['responseStatus']}")

    def test_domain_information_basic(self):
        """Test GET /api/{version}/objects/domain (basic)"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the domain info URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/domain"
            
            # Make direct API call using the client's session
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                self.current_domain_info = data
                
                # Extract domain information
                domain_info = {}
                if 'domain__v' in data:
                    domain = data['domain__v']
                    domain_info['domain_name'] = domain.get('domain_name__v', 'Unknown')
                    domain_info['domain_type'] = domain.get('domain_type__v', 'Unknown')
                    
                    # Track domain type
                    domain_type = domain.get('domain_type__v')
                    if domain_type and domain_type not in self.domain_types_found:
                        self.domain_types_found.append(domain_type)
                    
                    # Count vaults
                    vaults = domain.get('vaults__v', [])
                    domain_info['vault_count'] = len(vaults)
                    
                    # Analyze vault information
                    active_vaults = 0
                    inactive_vaults = 0
                    for vault in vaults:
                        vault_status = vault.get('vault_status__v', 'Unknown')
                        if vault_status == 'Active':
                            active_vaults += 1
                        elif vault_status == 'Inactive':
                            inactive_vaults += 1
                            
                        # Track vault families
                        if 'vault_family__v' in vault:
                            family = vault['vault_family__v']
                            family_name = family.get('name__v', 'Unknown')
                            if family_name not in [f['name'] for f in self.vault_families_found]:
                                self.vault_families_found.append({
                                    'name': family_name,
                                    'label': family.get('label__v', 'Unknown')
                                })
                    
                    domain_info['active_vaults'] = active_vaults
                    domain_info['inactive_vaults'] = inactive_vaults
                
                self.log_test(
                    "Domain Information (Basic)", True, response_time,
                    response_data=data,
                    endpoint="/objects/domain", method="GET"
                )
                
                print(f"        🏢 Domain: {domain_info.get('domain_name', 'Unknown')}")
                print(f"        🏷️ Type: {domain_info.get('domain_type', 'Unknown')}")
                print(f"        📦 Total Vaults: {domain_info.get('vault_count', 0)}")
                print(f"        ✅ Active: {domain_info.get('active_vaults', 0)}")
                print(f"        ❌ Inactive: {domain_info.get('inactive_vaults', 0)}")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "Domain Information (Basic)", False, response_time,
                    error_msg=error_msg,
                    endpoint="/objects/domain", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "Domain Information (Basic)", False, time.time() - start_time,
                error_msg=str(e), endpoint="/objects/domain", method="GET"
            )
            return None

    def test_domain_information_with_applications(self):
        """Test GET /api/{version}/objects/domain?include_application=true"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the domain info URL with application parameter
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/domain?include_application=true"
            
            # Make direct API call
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                
                # Extract application information
                app_info = {}
                if 'domain__v' in data:
                    domain = data['domain__v']
                    vaults = domain.get('vaults__v', [])
                    
                    total_applications = 0
                    vault_with_apps = 0
                    
                    for vault in vaults:
                        vault_apps = vault.get('vault_application__v', [])
                        if vault_apps:
                            vault_with_apps += 1
                            total_applications += len(vault_apps)
                            
                            # Track applications
                            for app in vault_apps:
                                app_name = app.get('name', 'Unknown')
                                app_label = app.get('label', 'Unknown')
                                if app_name not in [a['name'] for a in self.vault_applications_found]:
                                    self.vault_applications_found.append({
                                        'name': app_name,
                                        'label': app_label
                                    })
                    
                    app_info['vaults_with_applications'] = vault_with_apps
                    app_info['total_applications'] = total_applications
                    app_info['unique_applications'] = len(self.vault_applications_found)
                
                self.log_test(
                    "Domain Information (With Applications)", True, response_time,
                    response_data=data,
                    endpoint="/objects/domain?include_application=true", method="GET"
                )
                
                print(f"        📱 Vaults with Apps: {app_info.get('vaults_with_applications', 0)}")
                print(f"        📊 Total Applications: {app_info.get('total_applications', 0)}")
                print(f"        🆔 Unique Applications: {app_info.get('unique_applications', 0)}")
                
                # Show sample applications
                for app in self.vault_applications_found[:3]:
                    print(f"        📦 {app['name']}: {app['label']}")
                if len(self.vault_applications_found) > 3:
                    print(f"        ... and {len(self.vault_applications_found) - 3} more")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "Domain Information (With Applications)", False, response_time,
                    error_msg=error_msg,
                    endpoint="/objects/domain?include_application=true", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "Domain Information (With Applications)", False, time.time() - start_time,
                error_msg=str(e), endpoint="/objects/domain?include_application=true", method="GET"
            )
            return None

    def test_accessible_domains(self):
        """Test GET /api/{version}/objects/domains"""
        start_time = time.time()
        
        try:
            self.total_api_calls_tested += 1
            
            # Build the accessible domains URL
            base_url = self.client.vaultDNS.replace('https://', '').replace('http://', '')
            if not base_url.startswith('http'):
                base_url = f"https://{base_url}"
                
            url = f"{base_url}/api/v25.2/objects/domains"
            
            # Make direct API call
            headers = {
                'Authorization': self.client.sessionId,
                'Accept': 'application/json'
            }
            
            import requests
            response = requests.get(url, headers=headers)
            response_time = time.time() - start_time
            
            if response.status_code == 200:
                data = response.json()
                
                # Extract domains information
                domains = data.get('domains', [])
                self.accessible_domains = domains
                
                # Analyze domain types
                domain_stats = {}
                for domain in domains:
                    domain_type = domain.get('type', 'Unknown')
                    if domain_type in domain_stats:
                        domain_stats[domain_type] += 1
                    else:
                        domain_stats[domain_type] = 1
                        
                    # Track domain types
                    if domain_type not in self.domain_types_found:
                        self.domain_types_found.append(domain_type)
                
                self.log_test(
                    "Accessible Domains", True, response_time,
                    response_data=data,
                    endpoint="/objects/domains", method="GET"
                )
                
                print(f"        🌐 Accessible Domains: {len(domains)}")
                for domain in domains[:5]:  # Show first 5
                    name = domain.get('name', 'Unknown')
                    domain_type = domain.get('type', 'Unknown')
                    print(f"        🏢 {name}: {domain_type}")
                    
                if len(domains) > 5:
                    print(f"        ... and {len(domains) - 5} more domains")
                
                print(f"        📊 Domain Type Distribution: {domain_stats}")
                
                return data
                
            else:
                error_msg = f"HTTP {response.status_code}: {response.text}"
                self.log_test(
                    "Accessible Domains", False, response_time,
                    error_msg=error_msg,
                    endpoint="/objects/domains", method="GET",
                    status_code=response.status_code
                )
                return None
                
        except Exception as e:
            self.log_test(
                "Accessible Domains", False, time.time() - start_time,
                error_msg=str(e), endpoint="/objects/domains", method="GET"
            )
            return None
    
    def generate_summary(self):
        """Generate comprehensive test summary"""
        total_time = time.time() - self.start_time
        success_count = sum(1 for r in self.results if r.success)
        total_tests = len(self.results)
        
        print("\n" + "="*60)
        print("🏢 DOMAIN INFORMATION API TESTING - FINAL SUMMARY")
        print("="*60)
        
        print(f"\n📊 TEST SUMMARY")
        print("="*40)
        print(f"✅ SUCCESS: {success_count}")
        print(f"❌ FAILED: {total_tests - success_count}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.client.sessionId else '❌ Inactive'}")
        
        print(f"\n📊 DETAILED RESULTS")
        print("-" * 40)
        for i, result in enumerate(self.results, 1):
            status = "✅ PASS" if result.success else "❌ FAIL"
            print(f"{status} [{i:02d}]: {result.name} ({result.duration:.2f}s)")
            if result.success and result.response_data:
                if 'vault_id' in result.response_data:
                    print(f"        🏢 Vault ID: {result.response_data['vault_id']}")
            elif not result.success:
                print(f"        ❌ Error: {result.error_message}")
        
        print(f"\n🏆 FINAL STATISTICS")
        print("-" * 40)
        print(f"📈 Success Rate: {success_count}/{total_tests} ({(success_count/total_tests*100):.1f}%)")
        print(f"⏱️ Total Execution Time: {total_time:.2f}s")
        print(f"🌐 Accessible Domains: {len(self.accessible_domains)}")
        print(f"🏷️ Domain Types: {len(self.domain_types_found)}")
        print(f"📱 Vault Applications: {len(self.vault_applications_found)}")
        print(f"👥 Vault Families: {len(self.vault_families_found)}")
        print(f"📞 Total API Calls: {self.total_api_calls_tested}")
        
        if self.client.vaultDNS and self.client.sessionId:
            print(f"\n🏢 VAULT STATE")
            print("-" * 40)
            print(f"🔗 Connected to: {self.client.vaultDNS}")
            if hasattr(self.client, 'vaultId'):
                print(f"🆔 Vault ID: {self.client.vaultId}")
            if hasattr(self.client, 'vaultUserName'):
                print(f"👤 User: {self.client.vaultUserName}")
            print(f"🔐 Session: Active")
        
        print(f"\n🎯 API COVERAGE")
        print("-" * 40)
        print("✅ GET /objects/domain")
        print("✅ GET /objects/domain?include_application=true")
        print("✅ GET /objects/domains")
        
        print(f"\n🔍 DISCOVERED DOMAIN FEATURES")
        print("-" * 40)
        if self.accessible_domains:
            print(f"🌐 Accessible Domains: {len(self.accessible_domains)}")
            for domain in self.accessible_domains[:3]:
                print(f"   • {domain.get('name', 'Unknown')} ({domain.get('type', 'Unknown')})")
            if len(self.accessible_domains) > 3:
                print(f"   ... and {len(self.accessible_domains) - 3} more")
        
        if self.domain_types_found:
            print(f"🏷️ Domain Types: {', '.join(self.domain_types_found)}")
        
        if self.vault_applications_found:
            print(f"📱 Vault Applications: {len(self.vault_applications_found)}")
            for app in self.vault_applications_found[:3]:
                print(f"   • {app['name']}: {app['label']}")
            if len(self.vault_applications_found) > 3:
                print(f"   ... and {len(self.vault_applications_found) - 3} more")
        
        print(f"\n✅ Domain Information API testing completed successfully!")
        print(f"🏢 All operations were read-only and safe")
        print(f"📊 Comprehensive coverage of all domain information endpoints")
        
        # Export summary for continuation
        summary_data = {
            'section': 18,
            'name': 'Domain Information',
            'success_rate': f"{(success_count/total_tests*100):.1f}%",
            'total_tests': total_tests,
            'successful_tests': success_count,
            'total_time': f"{total_time:.2f}s",
            'accessible_domains': len(self.accessible_domains),
            'api_calls_tested': self.total_api_calls_tested
        }
        
        print(f"\n📦 Section summary exported for continuation: {summary_data['success_rate']} success rate")
        print("="*60)
        
        return summary_data

print("✅ Domain Information API Testing Framework loaded!")
print("🏢 Ready for comprehensive domain information testing")
print("🎯 Features: Domain details, vault information, application discovery, multi-domain access")

✅ Domain Information API Testing Framework loaded!
🏢 Ready for comprehensive domain information testing
🎯 Features: Domain details, vault information, application discovery, multi-domain access


In [2]:
# 🔐 Load credentials 
exec(open('/Users/mp/Documents/Code/VeevaTools/veevatools/veevavault/test_credentials.py').read())

# Extract credentials from TEST_VAULTS
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_DNS = vault_config["URL"]
USERNAME = vault_config["username"]
PASSWORD = vault_config["password"]

print("🔐 Credentials loaded successfully!")
print(f"🏢 Vault DNS: {VAULT_DNS}")
print(f"👤 Username: {USERNAME}")
print("✅ Ready for Domain Information API testing!")

🔐 Credentials loaded successfully!
🏢 Vault DNS: https://vv-consulting-michael-mastermind.veevavault.com/
👤 Username: veevatools@vv-consulting.com
✅ Ready for Domain Information API testing!


In [3]:
# 🚀 Initialize Domain Information API Testing Framework
from veevavault.client.vault_client import VaultClient

# Create client instance (reuse the working client if available)
if 'security_client' in locals() and security_client.sessionId:
    print("🔄 Reusing authenticated client from Section 17...")
    domain_client = security_client  # Reuse the working authenticated client
else:
    print("🆕 Creating new client instance...")
    domain_client = VaultClient()
    domain_client.vaultDNS = VAULT_DNS

# Initialize Domain Information tester
domain_tester = DomainInformationAPITester(domain_client)

print("🚀 Domain Information API Testing Framework initialized!")
print("🏢 Ready to test VeevaVault Domain Information capabilities")
print("🎯 Framework features:")
print("   ✅ Domain information retrieval")
print("   ✅ Vault listing and analysis")
print("   ✅ Application details discovery")
print("   ✅ Multi-domain access testing")
print("   ✅ Domain type classification")

🆕 Creating new client instance...
🚀 Domain Information API Testing Framework initialized!
🏢 Ready to test VeevaVault Domain Information capabilities
🎯 Framework features:
   ✅ Domain information retrieval
   ✅ Vault listing and analysis
   ✅ Application details discovery
   ✅ Multi-domain access testing
   ✅ Domain type classification


In [4]:
# 🔑 Authenticate with VeevaVault (if needed)
if not domain_client.sessionId:
    print("🔑 Authenticating with VeevaVault...")

    start_time = time.time()
    try:
        # Authenticate using the correct parameters
        auth_response = domain_client.authenticate(
            vaultURL=VAULT_DNS,
            vaultUserName=USERNAME,
            vaultPassword=PASSWORD,
            if_return=True
        )
        auth_duration = time.time() - start_time
        
        # VaultClient returns sessionId on success instead of responseStatus
        if auth_response and auth_response.get('sessionId'):
            vault_id = auth_response.get('vaultId')
            session_id = auth_response.get('sessionId')
            
            print(f"✅ Authentication successful!")
            print(f"🏢 Vault ID: {vault_id}")
            print(f"🔐 Session established: {session_id[:20]}...")
            
            # Initialize query services now that we're authenticated
            domain_tester.init_services()
            
            # Log successful authentication
            domain_tester.log_test(
                "Authentication", True, auth_duration,
                response_data={'vault_id': vault_id, 'session_id': session_id},
                endpoint="/auth", method="POST"
            )
            
        else:
            error_msg = "Authentication failed - no session ID returned"
            print(f"❌ {error_msg}")
            if auth_response:
                print(f"Response: {auth_response}")
            domain_tester.log_test(
                "Authentication", False, auth_duration,
                error_msg=error_msg, endpoint="/auth", method="POST"
            )
            
    except Exception as e:
        auth_duration = time.time() - start_time
        error_msg = f"Authentication exception: {str(e)}"
        print(f"❌ {error_msg}")
        domain_tester.log_test(
            "Authentication", False, auth_duration,
            error_msg=error_msg, endpoint="/auth", method="POST"
        )
else:
    print("✅ Using existing authenticated session from Section 17!")
    print(f"🏢 Vault ID: {domain_client.vaultId}")
    print(f"🔐 Session: {domain_client.sessionId[:20]}...")
    
    # Initialize services with existing session
    domain_tester.init_services()
    
    # Log reused authentication
    domain_tester.log_test(
        "Authentication (Reused)", True, 0.0,
        response_data={'vault_id': domain_client.vaultId, 'session_reused': True},
        endpoint="/auth", method="REUSE"
    )

print(f"\n🎯 Ready to test Domain Information APIs!")

🔑 Authenticating with VeevaVault...
✅ Authentication successful!
🏢 Vault ID: 92425
🔐 Session established: FF55C5DC28CC5FC1AB29...
✅ PASS [01]: Authentication (0.60s)

🎯 Ready to test Domain Information APIs!


In [5]:
# 🏢 Test 1: Domain Information (Basic)
print("🔍 Testing Domain Information (Basic)...")
print("📋 GET /api/v25.2/objects/domain")

basic_domain_result = domain_tester.test_domain_information_basic()

print(f"\n✅ Basic domain information testing completed!")
if basic_domain_result:
    print(f"🏢 Domain information retrieved and analyzed")
else:
    print(f"⚠️ Basic domain information failed - check vault permissions")

🔍 Testing Domain Information (Basic)...
📋 GET /api/v25.2/objects/domain
✅ PASS [02]: Domain Information (Basic) (0.29s)
        📊 Status: SUCCESS
        🏢 Domain: vv-consulting
        🏷️ Type: Sandbox
        📦 Total Vaults: 1
        ✅ Active: 1
        ❌ Inactive: 0

✅ Basic domain information testing completed!
🏢 Domain information retrieved and analyzed


In [6]:
# 📱 Test 2: Domain Information (With Applications)
print("🔍 Testing Domain Information (With Applications)...")
print("📋 GET /api/v25.2/objects/domain?include_application=true")

app_domain_result = domain_tester.test_domain_information_with_applications()

print(f"\n✅ Domain information with applications testing completed!")
if app_domain_result:
    print(f"📱 Application details retrieved: {len(domain_tester.vault_applications_found)} unique apps")
else:
    print(f"⚠️ Application information retrieval failed - check vault permissions")

🔍 Testing Domain Information (With Applications)...
📋 GET /api/v25.2/objects/domain?include_application=true
✅ PASS [03]: Domain Information (With Applications) (0.29s)
        📊 Status: SUCCESS
        📱 Vaults with Apps: 1
        📊 Total Applications: 1
        🆔 Unique Applications: 1
        📦 platform_platform__sys: Platform

✅ Domain information with applications testing completed!
📱 Application details retrieved: 1 unique apps


In [7]:
# 🌐 Test 3: Accessible Domains
print("🔍 Testing Accessible Domains...")
print("📋 GET /api/v25.2/objects/domains")

domains_result = domain_tester.test_accessible_domains()

print(f"\n✅ Accessible domains testing completed!")
if domains_result:
    print(f"🌐 Accessible domains found: {len(domain_tester.accessible_domains)}")
    print(f"🏷️ Domain types discovered: {domain_tester.domain_types_found}")
else:
    print(f"⚠️ Accessible domains retrieval failed - check vault permissions")

🔍 Testing Accessible Domains...
📋 GET /api/v25.2/objects/domains
✅ PASS [04]: Accessible Domains (0.19s)
        📊 Status: SUCCESS
        🌐 Accessible Domains: 75
        🏢 animalhealth-tf.com: Demo
        🏢 ap-cdp.com: Sandbox
        🏢 apachiring.com: Sandbox
        🏢 asia-cdms.com: Demo
        🏢 cdms-cdash-library.com: Sandbox
        ... and 70 more domains
        📊 Domain Type Distribution: {'Demo': 27, 'Sandbox': 46, 'Production': 1, 'Template': 1}

✅ Accessible domains testing completed!
🌐 Accessible domains found: 75
🏷️ Domain types discovered: ['Sandbox', 'Demo', 'Production', 'Template']


In [8]:
# 📊 Generate Comprehensive Test Summary
print("📊 Generating comprehensive Domain Information test summary...")

domain_summary = domain_tester.generate_summary()

# Store results for next section
section_18_results = {
    'domain_tester': domain_tester,
    'summary': domain_summary,
    'client': domain_client,
    'current_domain_info': domain_tester.current_domain_info,
    'accessible_domains': domain_tester.accessible_domains,
    'vault_applications': domain_tester.vault_applications_found
}

print(f"\n🎯 Section 18 (Domain Information) completed successfully!")
print(f"🏢 All domain information endpoints tested!")
print(f"📋 Ready to continue to Section 19...")

📊 Generating comprehensive Domain Information test summary...

🏢 DOMAIN INFORMATION API TESTING - FINAL SUMMARY

📊 TEST SUMMARY
✅ SUCCESS: 4
❌ FAILED: 0
⏱️ Total time: 18.08s
🔐 Session: ✅ Active

📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.60s)
        🏢 Vault ID: 92425
✅ PASS [02]: Domain Information (Basic) (0.29s)
✅ PASS [03]: Domain Information (With Applications) (0.29s)
✅ PASS [04]: Accessible Domains (0.19s)

🏆 FINAL STATISTICS
----------------------------------------
📈 Success Rate: 4/4 (100.0%)
⏱️ Total Execution Time: 18.08s
🌐 Accessible Domains: 75
🏷️ Domain Types: 4
📱 Vault Applications: 1
👥 Vault Families: 1
📞 Total API Calls: 3

🏢 VAULT STATE
----------------------------------------
🔗 Connected to: vv-consulting-michael-mastermind.veevavault.com
🆔 Vault ID: 92425
👤 User: veevatools@vv-consulting.com
🔐 Session: Active

🎯 API COVERAGE
----------------------------------------
✅ GET /objects/domain
✅ GET /objects/domain?include_a

# Baseline Veeva Vault API Testing Template

This is a minimal baseline template for Veeva Vault API testing that contains only the essential components needed by all section-specific test notebooks.

## Template Purpose

This template should be **copied and customized** for testing each of the 38 API documentation sections:

**Core APIs (01-10):**
- 01_authentication.md → 01_AuthenticationTests.ipynb
- 02_direct_data.md → 02_DirectDataTests.ipynb  
- 03_vault_query_language_vql.md → 03_VQLTests.ipynb
- 04_metadata_definition_language_mdl.md → 04_MDLTests.ipynb
- 05_documents.md → 05_DocumentTests.ipynb
- 06_binders.md → 06_BinderTests.ipynb
- 07_vault_objects.md → 07_ObjectTests.ipynb
- 08_document_and_binder_roles.md → 08_RoleTests.ipynb
- 09_workflows.md → 09_WorkflowTests.ipynb
- 10_document_lifecycle_workflows.md → 10_DocLifecycleTests.ipynb

**And 28 more sections...**

## What This Template Provides

✅ **Essential imports** - VaultClient, Authentication, Query services  
✅ **Credential loading** - from test_credentials.py  
✅ **Authentication framework** - with session management  
✅ **Test result tracking** - structured logging and timing  
✅ **Error handling** - comprehensive exception management  

## What to Add in Section-Specific Notebooks

➕ **Additional service imports** - specific to the API section  
➕ **Section-specific tests** - based on api_docs/sections/*.md  
➕ **Endpoint validation** - following notebook_tests/*.md  
➕ **Custom test cases** - for specific use cases  

## Prerequisites

- Active .venv with veevatools installed
- Valid test_credentials.py with vault access
- VeevaVault client and services properly configured

In [2]:
# Core imports
import sys
import os
import json
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

# VeevaVault Core Client
from veevavault.client import VaultClient

# Essential Services for baseline template
from veevavault.services.authentication import AuthenticationService
from veevavault.services.queries import QueryService

print("✅ Successfully imported essential VeevaVault services")
print(f"🐍 Python version: {sys.version}")
print(f"📦 Virtual environment active: {sys.prefix != sys.base_prefix}")
print("📚 Baseline template loaded with core services only")

✅ Successfully imported essential VeevaVault services
🐍 Python version: 3.11.11 (main, Jan 30 2025, 20:55:05) [Clang 16.0.0 (clang-1600.0.26.6)]
📦 Virtual environment active: True
📚 Baseline template loaded with core services only


## Configuration and Credentials

Load vault credentials and configure the testing environment.

**Required:** Ensure `test_credentials.py` exists with valid vault access credentials.

In [3]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    
    # Extract credentials
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print(f"Available vaults: {list(TEST_VAULTS.keys()) if 'TEST_VAULTS' in locals() else 'None'}")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## Baseline Testing Framework

Minimal testing framework with essential components for all API testing.

In [4]:
class APITestResult:
    """Structured container for API test results."""
    
    def __init__(self, test_name: str, status: str, section: str = None, 
                 endpoint: str = None, details: Dict[str, Any] = None, 
                 error: str = None, response_data: Any = None,
                 execution_time: float = None):
        self.timestamp = datetime.now().isoformat()
        self.test_name = test_name
        self.status = status  # SUCCESS, FAILURE, WARNING, SKIPPED
        self.section = section  # API section (01-38)
        self.endpoint = endpoint  # Specific API endpoint
        self.details = details or {}
        self.error = error
        self.response_data = response_data
        self.execution_time = execution_time

class BaselineAPITester:
    """Minimal baseline API testing framework."""
    
    def __init__(self, vault_client: VaultClient):
        self.vault_client = vault_client
        self.test_results: List[APITestResult] = []
        self.session_active = False
        
        # Initialize essential services
        self.auth_service = AuthenticationService(vault_client)
        self.query_service = QueryService(vault_client)
        
    def log_test_result(self, test_name: str, status: str, section: str = None,
                       endpoint: str = None, details: Dict[str, Any] = None,
                       error: str = None, response_data: Any = None,
                       execution_time: float = None):
        """Log a test result."""
        result = APITestResult(
            test_name=test_name, status=status, section=section, endpoint=endpoint,
            details=details, error=error, response_data=response_data, execution_time=execution_time
        )
        self.test_results.append(result)
        
        # Print status with emoji
        status_emoji = {'SUCCESS': '✅', 'FAILURE': '❌', 'WARNING': '⚠️', 'SKIPPED': '⏭️'}
        section_info = f" [{section}]" if section else ""
        time_info = f" ({execution_time:.2f}s)" if execution_time else ""
        
        print(f"{status_emoji.get(status, '❓')} {test_name}{section_info}: {status}{time_info}")
        if error:
            print(f"   ❌ Error: {error}")
        if details:
            print(f"   📋 Details: {details}")
    
    def test_with_timing(self, test_func, test_name: str, section: str = None, 
                        endpoint: str = None, **kwargs):
        """Execute a test function with timing and error handling."""
        start_time = time.time()
        
        try:
            result = test_func(**kwargs)
            execution_time = time.time() - start_time
            
            if isinstance(result, dict) and result.get('responseStatus') == 'SUCCESS':
                self.log_test_result(test_name, 'SUCCESS', section, endpoint, 
                                   response_data=result, execution_time=execution_time)
            elif isinstance(result, dict) and result.get('responseStatus') == 'FAILURE':
                error_msg = result.get('errors', [{}])[0].get('message', 'API returned FAILURE')
                self.log_test_result(test_name, 'FAILURE', section, endpoint, 
                                   error=error_msg, response_data=result, execution_time=execution_time)
            else:
                self.log_test_result(test_name, 'SUCCESS', section, endpoint, 
                                   response_data=result, execution_time=execution_time)
            return result
            
        except Exception as e:
            execution_time = time.time() - start_time
            self.log_test_result(test_name, 'FAILURE', section, endpoint, 
                               error=str(e), execution_time=execution_time)
            return None
    
    def authenticate(self) -> bool:
        """Perform authentication and setup session."""
        auth_response = self.test_with_timing(
            test_func=self._do_authentication,
            test_name="Authentication",
            section="01",
            endpoint="/api/auth"
        )
        return auth_response is not None and auth_response.get('responseStatus') == 'SUCCESS'
    
    def _do_authentication(self):
        """Internal authentication implementation."""
        auth_response = self.auth_service.authenticate_with_username_password(
            username=VAULT_USERNAME,
            password=VAULT_PASSWORD,
            vaultDNS=VAULT_DNS
        )
        
        if auth_response.get('responseStatus') == 'SUCCESS':
            session_id = auth_response.get('sessionId')
            user_id = auth_response.get('userId')
            vault_id = auth_response.get('vaultId')
            
            # Update client with session info
            self.vault_client.sessionId = session_id
            self.vault_client.session_id = session_id
            self.vault_client.vaultId = vault_id
            self.vault_client.vaultDNS = VAULT_DNS
            
            self.session_active = True
        else:
            self.session_active = False
            
        return auth_response
    
    def print_summary(self):
        """Print test summary."""
        if not self.test_results:
            print("No tests executed yet.")
            return
            
        df = pd.DataFrame([{
            'test_name': r.test_name,
            'status': r.status,
            'section': r.section,
            'execution_time': r.execution_time
        } for r in self.test_results])
        
        status_counts = df['status'].value_counts()
        print("\n📊 TEST SUMMARY")
        print("=" * 40)
        for status, count in status_counts.items():
            emoji = {'SUCCESS': '✅', 'FAILURE': '❌', 'WARNING': '⚠️', 'SKIPPED': '⏭️'}.get(status, '❓')
            print(f"{emoji} {status}: {count}")
        
        if 'execution_time' in df.columns and df['execution_time'].notna().any():
            print(f"⏱️ Total time: {df['execution_time'].sum():.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_active else '❌ Inactive'}")

print("✅ Baseline API testing framework initialized")

✅ Baseline API testing framework initialized


## Client Initialization and Authentication

Initialize the VaultClient and perform authentication to establish the testing session.

In [5]:
# Initialize the main vault client
print("🔧 Initializing VeevaVault client...")

vault_client = VaultClient()
vault_client.vaultURL = VAULT_URL
vault_client.vaultDNS = VAULT_DNS
vault_client.vaultUserName = VAULT_USERNAME
vault_client.vaultPassword = VAULT_PASSWORD

print(f"✅ VeevaVault client initialized")
print(f"🔗 Vault URL: {vault_client.vaultURL}")
print(f"📡 Vault DNS: {vault_client.vaultDNS}")
print(f"👤 Username: {vault_client.vaultUserName}")

# Initialize the baseline test framework
print("🔧 Initializing baseline test framework...")
baseline_tester = BaselineAPITester(vault_client)

print(f"✅ Baseline framework ready")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com/
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing baseline test framework...
✅ Baseline framework ready


In [6]:
# Perform authentication 
print("🔐 Performing authentication...")

auth_success = baseline_tester.authenticate()

if auth_success:
    print(f"\n🎉 Authentication Successful!")
    print(f"🆔 Session ID: {vault_client.sessionId[:20]}..." if vault_client.sessionId else "No session ID")
    print(f"🏢 Vault ID: {vault_client.vaultId}")
    print("✅ Baseline template validated - ready for section-specific testing!")
else:
    print("❌ Authentication failed! Check credentials and try again.")

# Show summary
baseline_tester.print_summary()

🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.50s)

🎉 Authentication Successful!
🆔 Session ID: EFFDF861DBECA76326C1...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.50s
🔐 Session: ✅ Active
✅ Authentication [01]: SUCCESS (0.50s)

🎉 Authentication Successful!
🆔 Session ID: EFFDF861DBECA76326C1...
🏢 Vault ID: 92425
✅ Baseline template validated - ready for section-specific testing!

📊 TEST SUMMARY
✅ SUCCESS: 1
⏱️ Total time: 0.50s
🔐 Session: ✅ Active


## Session Management and Next Steps

Basic session utilities and guidance for creating section-specific test notebooks.

### Available Functions:
- `get_session_info()` - Display current session status
- `end_session()` - Clean session logout
- `baseline_tester.print_summary()` - Show test results

### Create Section-Specific Notebooks:

1. **Copy this baseline template** 
2. **Import additional services** for the specific API section
3. **Add test methods** based on `api_docs/sections/XX_*.md`
4. **Follow test plans** in `notebook_tests/XX_*.md`
5. **Run comprehensive testing** for the section

**✅ Baseline template validated and ready for use!**

In [8]:
# Session management utilities
def get_session_info():
    """Get current session information."""
    print("🔍 SESSION INFO")
    print("="*30)
    print(f"Active: {'✅ Yes' if baseline_tester.session_active else '❌ No'}")
    print(f"Session ID: {vault_client.sessionId[:20] + '...' if vault_client.sessionId else 'None'}")
    print(f"Vault ID: {vault_client.vaultId or 'None'}")
    print(f"Tests Run: {len(baseline_tester.test_results)}")

def end_session():
    """End the current session."""
    if hasattr(baseline_tester.auth_service, 'logout') and baseline_tester.session_active:
        try:
            baseline_tester.auth_service.logout()
            baseline_tester.session_active = False
            vault_client.sessionId = None
            print("✅ Session ended")
        except Exception as e:
            print(f"❌ Logout failed: {str(e)}")
    else:
        print("⚠️ No active session or logout not available")

print("✅ Session utilities ready: get_session_info(), end_session()")

# Display current session
get_session_info()

✅ Session utilities ready: get_session_info(), end_session()
🔍 SESSION INFO
Active: ✅ Yes
Session ID: EFFDF861DBECA76326C1...
Vault ID: 92425
Tests Run: 1
